## 2 Layered Neural Network

In [ ]:
# Backprop on the Seeds Dataset
from random import seed
from random import randrange
from random import random
from random import gauss
from random import uniform
from random import shuffle
from csv import reader
from math import exp
import numpy as np
import matplotlib.pyplot as plt 
from statistics import stdev
# Load a CSV file
def load_csv(filename):
	dataset = list()
	with open(filename, 'r') as file:
		csv_reader = reader(file)
		for row in csv_reader:
			if not row:
				continue #[[],[]]
			dataset.append(row)
	return dataset
 
# Convert string column to float
def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column].strip())
 
# Convert string column to integer
def str_column_to_int(dataset, column):
	class_values = [row[column] for row in dataset]
	unique = set(class_values)
	lookup = dict()
	for i, value in enumerate(unique):
		lookup[value] = i
	for row in dataset:
		row[column] = lookup[row[column]]
	return lookup
 
# Find the min and max values for each column
def dataset_minmax(dataset):
	minmax = list()
	stats = [[min(column), max(column)] for column in zip(*dataset)]
	return stats
 
# Rescale dataset columns to the range 0-1
def normalize_dataset(dataset, minmax):
	for row in dataset:
		for i in range(len(row)-1):
			row[i] = (float(row[i]) - float(minmax[i][0])) / (float(minmax[i][1]) - float(minmax[i][0]))
 
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0

def evaluate_algorithm(dataset, algorithm, *args):
  shuffle(dataset)
  train_set = dataset[:int(0.7*len(dataset))]
  test_set = dataset[int(0.7*len(dataset))+1:]
  scores = list()
  predicted = algorithm(train_set, test_set, *args)
  print(len(predicted))
  actual = [row[-1] for row in test_set]
  print(len(actual))
  accuracy = accuracy_metric(actual, predicted)
  scores.append(accuracy)
  return scores
 
# Calculate neuron activation for an input
def activate(weights, inputs):
	activation = weights[-1]
	for i in range(len(weights)-1):
		activation += weights[i] * inputs[i]
	return activation
 
# Transfer neuron activation
def transfer_sigmoid(activation):
    return 1.0 / (1.0 + exp(-activation))#sigmoid


# Forward propagate input to a network output
def forward_propagate(network, row):
	inputs = row
	for layer in network:
		new_inputs = []
		for neuron in layer:
			activation = activate(neuron['weights'],inputs)
			neuron['output'] = transfer_sigmoid(activation)
			new_inputs.append(neuron['output'])
		inputs = new_inputs
	return inputs
 
# Calculate the derivative of an neuron output
def transfer_derivative_sigmoid(output):
	return output * (1.0 - output)

def transfer_derivative_relu(output):
	if output<=0:
		return 0
	else:
		return 1
 
# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
	for i in reversed(range(len(network))):
		layer = network[i]
		errors = list()
		if i != len(network)-1:
			for j in range(len(layer)):
				error = 0.0
				for neuron in network[i + 1]:
					error += (neuron['weights'][j] * neuron['delta'])
				errors.append(error)
		else:
			for j in range(len(layer)):
				neuron = layer[j]
				errors.append(expected[j] - neuron['output'])
		for j in range(len(layer)):
			neuron = layer[j]
			neuron['delta'] = errors[j] * transfer_derivative_sigmoid(neuron['output'])
 
# Update network weights with error
def update_weights(network, row, l_rate):
	for i in range(len(network)):
		inputs = row[:-1]
		if i != 0:
			inputs = [neuron['output'] for neuron in network[i - 1]]
		for neuron in network[i]:
			for j in range(len(inputs)):
				neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
			neuron['weights'][-1] += l_rate * neuron['delta']

  
def train_network(network, train, l_rate, n_epoch, n_outputs):
  errors=list()
  sum_error=None
  acc_values=list()
  iter=list()
  for epoch in range(n_epoch):
    sum_error=0
    act=list()
    for r in train:
      act.append(r[-1])
    for row in train:
      outputs=forward_propagate(network,row)
      expected=[0 for i in range(n_outputs)]
      expected[row[-1]] = 1
      sum_error+=np.linalg.norm(np.array([x1 - x2 for (x1, x2) in zip(outputs, expected)]))
      backward_propagate_error(network, expected)
      update_weights(network, row, l_rate)
    errors.append(sum_error)
    if epoch%50==0:
      pred=list()
      for row in train:
        p=predict(network,row)
        # print(len(act))
        # print(len(p))
        pred.append(p)
      acc_values.append(accuracy_metric(act,pred))
      iter.append(epoch)
  x = [i for i in range(1,len(errors)+1)]
  y = errors
  print('____TRAIN DATA____')
  print('Average Cost:',float(sum(errors))/float(len(errors)))
  print('Average Cost:',float(sum(acc_values))/float(len(acc_values)))
  plt.plot(x, y)
  plt.xlabel('Epochs')
  plt.ylabel('Error(Mean Squared)')
  plt.title('Epochs vs Error')
  plt.show()
  # print(acc_values)
  # print(iter)
  plt.plot(iter,acc_values)
  plt.xlabel('Epoches')
  plt.ylabel('Accuracies')
  plt.title('Epochs vs Accuracies')
  plt.show()


# Initialize a network
def initialize_network(n_inputs, n_hidden, n_outputs):
  # print(n_outputs)
  network = list()
  hidden_layer_1 = [{'weights':[uniform(0,1) for i in range(n_inputs + 1)]} for i in range(n_hidden[0])]
  network.append(hidden_layer_1)
  hidden_layer_2 = [{'weights':[uniform(0,1) for i in range(n_hidden[0] + 1)]} for i in range(n_hidden[1])]
  network.append(hidden_layer_2)
  output_layer = [{'weights':[uniform(0,1) for i in range(n_hidden[1] + 1)]} for i in range(n_outputs)]
  network.append(output_layer)
  return network
 
# Make a prediction with a network
def predict(network, row):
	outputs = forward_propagate(network, row)
	return outputs.index(max(outputs))
 
# Backpropagation Algorithm With Stochastic Gradient Descent
def back_propagation(train, test, l_rate, n_epoch, n_hidden):
	n_inputs = len(train[0]) - 1
	n_outputs = len(set([row[-1] for row in train]))
	network = initialize_network(n_inputs, n_hidden, n_outputs)
	train_network(network, train, l_rate, n_epoch, n_outputs)
	predictions = list()
	for row in test:
		prediction = predict(network, row)
		predictions.append(prediction)
	return(predictions)
 
# Test Backprop on Seeds dataset
seed(1)
# load and prepare data
filename = 'dataset_NN.csv'
dataset = load_csv(filename)
# print(dataset)
dataset.pop(0)
for i in range(len(dataset[0])-1):
	str_column_to_float(dataset, i)
# # convert class column to integers
str_column_to_int(dataset, len(dataset[0])-1)
# normalize input variables

minmax = dataset_minmax(dataset)
# print(minmax)
normalize_dataset(dataset, minmax)
# evaluate algorithm
l_rate = 0.01
n_epoch = 5000 #500
n_hidden = [5,5]#list of number of neurons in each hidden layer
accuracies = evaluate_algorithm(dataset, back_propagation,l_rate, n_epoch, n_hidden)
# print('Accuracies: %s' % accuracies)
print('____TEST DATA____')
print('Average Accuracy: %.3f%%' % (sum(accuracies)/float(len(accuracies))))

## 1 Layered Neural Network

In [ ]:
# Backprop on the Seeds Dataset
from random import seed
from random import randrange
from random import random
from csv import reader
from math import exp
 
# Load a CSV file
def load_csv(filename):
	dataset = list()
	with open(filename, 'r') as file:
		csv_reader = reader(file)
		for row in csv_reader:
			if not row:
				continue
			dataset.append(row)
	return dataset
 
# Convert string column to float
def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column].strip())
 
# Convert string column to integer
def str_column_to_int(dataset, column):
	class_values = [row[column] for row in dataset]
	unique = set(class_values)
	lookup = dict()
	for i, value in enumerate(unique):
		lookup[value] = i
	for row in dataset:
		row[column] = lookup[row[column]]
	return lookup
 
# Find the min and max values for each column
def dataset_minmax(dataset):
	minmax = list()
	stats = [[min(column), max(column)] for column in zip(*dataset)]
	return stats
 
# Rescale dataset columns to the range 0-1
def normalize_dataset(dataset, minmax):
	for row in dataset:
		for i in range(len(row)-1):
			row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])
 
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0

def evaluate_algorithm(dataset, algorithm, n_folds, *args):
  shuffle(dataset)
  train_set = dataset[:int(0.7*len(dataset))]
  test_set = dataset[int(0.7*len(dataset))+1:]
  # print(train_set)
  scores = list()
  predicted = algorithm(train_set, test_set, *args)
  # print(len(predicted))
  actual = [row[-1] for row in test_set]
  # print(len(actual))
  accuracy = accuracy_metric(actual, predicted)
  scores.append(accuracy)
  return scores
# Calculate neuron activation for an input
def activate(weights, inputs):
	activation = weights[-1]
	for i in range(len(weights)-1):
		activation += weights[i] * inputs[i]
	return activation
 
# Transfer neuron activation
def transfer(activation):
	return 1.0 / (1.0 + exp(-activation))
 
# Forward propagate input to a network output
def forward_propagate(network, row):
	inputs = row
	for layer in network:
		new_inputs = []
		for neuron in layer:
			activation = activate(neuron['weights'], inputs)
			neuron['output'] = transfer(activation)
			new_inputs.append(neuron['output'])
		inputs = new_inputs
	return inputs
 
# Calculate the derivative of an neuron output
def transfer_derivative(output):
	return output * (1.0 - output)
 
# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
	for i in reversed(range(len(network))):
		layer = network[i]
		errors = list()
		if i != len(network)-1:
			for j in range(len(layer)):
				error = 0.0
				for neuron in network[i + 1]:
					error += (neuron['weights'][j] * neuron['delta'])
				errors.append(error)
		else:
			for j in range(len(layer)):
				neuron = layer[j]
				errors.append(expected[j] - neuron['output'])
		for j in range(len(layer)):
			neuron = layer[j]
			neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])
 
# Update network weights with error
def update_weights(network, row, l_rate):
	for i in range(len(network)):
		inputs = row[:-1]
		if i != 0:
			inputs = [neuron['output'] for neuron in network[i - 1]]
		for neuron in network[i]:
			for j in range(len(inputs)):
				neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
			neuron['weights'][-1] += l_rate * neuron['delta']

def train_network(network, train, l_rate, n_epoch, n_outputs):
  errors=list()
  sum_error=0
  acc_values=list()
  iter=list()
  for epoch in range(n_epoch):
    act=list()
    for r in train:
      act.append(r[-1])
    for row in train:
      outputs=forward_propagate(network, row)
      expected=[0 for i in range(n_outputs)]
      expected[row[-1]] = 1
      sum_error=np.linalg.norm(np.array([x1 - x2 for (x1, x2) in zip(outputs, expected)]))
      backward_propagate_error(network, expected)
      update_weights(network, row, l_rate)
    errors.append(sum_error)
    if epoch%50==0:
      pred=list()
      for row in train:
        p=predict(network,row)
        # print(len(act))
        # print(len(p))
        pred.append(p)
      acc_values.append(accuracy_metric(act,pred))
      iter.append(epoch)
  x = [i for i in range(1,len(errors)+1)]
  y = errors
  print('____TRAIN DATA____')
  print('Average Cost:',float(sum(errors))/float(len(errors)))
  print('Average Accuracy:',float(sum(acc_values))/float(len(acc_values)))
  plt.plot(x, y)
  plt.xlabel('Epochs')
  plt.ylabel('Error(Mean Squared)')
  plt.title('Epochs vs Error')
  plt.show()
  # print(acc_values)
  # print(iter)
  plt.plot(iter,acc_values)
  plt.xlabel('Epoches')
  plt.ylabel('Accuracies')
  plt.title('Epochs vs Accuracies')
  plt.show()

# Initialize a network
def initialize_network(n_inputs, n_hidden, n_outputs):
	network = list()
	hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
	network.append(hidden_layer)
	output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
	network.append(output_layer)
	return network
 
# Make a prediction with a network
def predict(network, row):
	outputs = forward_propagate(network, row)
	return outputs.index(max(outputs))
 
# Backpropagation Algorithm With Stochastic Gradient Descent
def back_propagation(train, test, l_rate, n_epoch, n_hidden):
	n_inputs = len(train[0]) - 1
	n_outputs = len(set([row[-1] for row in train]))
	network = initialize_network(n_inputs, n_hidden, n_outputs)
	train_network(network, train, l_rate, n_epoch, n_outputs)
	predictions = list()
	for row in test:
		prediction = predict(network, row)
		predictions.append(prediction)
	return predictions
 
# Test Backprop on Seeds dataset
seed(1)
# load and prepare data
filename = 'dataset_NN.csv'
dataset = load_csv(filename)
dataset.pop(0)
for i in range(len(dataset[0])-1):
	str_column_to_float(dataset, i)
# # convert class column to integers
str_column_to_int(dataset, len(dataset[0])-1)
# # normalize input variables
minmax = dataset_minmax(dataset)
normalize_dataset(dataset, minmax)
# evaluate algorithm
n_folds = 5
l_rate = 0.9
n_epoch = 5000
n_hidden = 8
scores = evaluate_algorithm(dataset, back_propagation, n_folds, l_rate, n_epoch, n_hidden)
# print('Scores: %s' % scores)
print('____TEST DATA____')
print('Average Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))